#Выбор данных

#Датасеты
- **Датасет:** [Stroke Prediction Dataset](https://www.kaggle.com/datasets/jawairia123/stroke-prediction-dataset)

 **Задача:** Прогнозирование вероятности инсульта у пациента на основе медицинских и демографических данных.

- **Датасет:** [Python Learning & Exam Performance Dataset](https://www.kaggle.com/datasets/emonsharkar/python-learning-and-exam-performance-dataset)

 **Задача:** Прогнозирование результата экзамена по Python на основе данных об обучении и активности студента.

#Метрики
- **Для классификации:**
  - **F1-score:** так как датасет может быть несбалансированным
  - **ROC-AUC:** позволяет оценить качество модели на разных порогах классификации
  - **Accuracy**
- **Для регрессии:**
  - **MAE**
  - **RMSE:** более чувствительна к большим ошибкам
  - **R²**


#Создание бейзлайна и оценка качества


In [171]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

Аналогично ЛР1

In [168]:
df_stroke = pd.read_csv('/content/healthcare-dataset-stroke-data.csv')
df_stroke = df_stroke.drop(columns=['id'])
df_exam = pd.read_csv('/content/python_learning_exam_performance.csv')
df_exam = df_exam.drop(columns=['student_id'])

df_stroke['bmi'] = df_stroke['bmi'].fillna(df_stroke['bmi'].median())

X_stroke = df_stroke.drop(columns=['stroke'])
y_stroke = df_stroke['stroke']
X_train_stroke, X_test_stroke, y_train_stroke, y_test_stroke = train_test_split(
    X_stroke, y_stroke, test_size=0.2, random_state=42, stratify=y_stroke
)

cat_cols_stroke = X_train_stroke.select_dtypes(include=['object']).columns.tolist()
num_cols_stroke = X_train_stroke.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_stroke = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols_stroke),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols_stroke)
    ]
)

X_train_stroke_processed = preprocessor_stroke.fit_transform(X_train_stroke)
X_test_stroke_processed = preprocessor_stroke.transform(X_test_stroke)
df_exam['prior_programming_experience'] = df_exam['prior_programming_experience'].fillna('No')

X_exam = df_exam.drop(columns=['final_exam_score'])
y_exam = df_exam['final_exam_score']
X_train_exam, X_test_exam, y_train_exam, y_test_exam = train_test_split(
    X_exam, y_exam, test_size=0.2, random_state=42
)

cat_cols_exam = X_train_exam.select_dtypes(include=['object']).columns.tolist()
num_cols_exam = X_train_exam.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_exam = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols_exam),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols_exam)
    ]
)

X_train_exam_processed = preprocessor_exam.fit_transform(X_train_exam)
X_test_exam_processed = preprocessor_exam.transform(X_test_exam)

Бейзлайн

In [172]:
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_stroke_processed, y_train_stroke)
y_pred_stroke_log = log_reg.predict(X_test_stroke_processed)
y_pred_proba_stroke_log = log_reg.predict_proba(X_test_stroke_processed)[:, 1]

lin_reg = LinearRegression()
lin_reg.fit(X_train_exam_processed, y_train_exam)
y_pred_exam_lin = lin_reg.predict(X_test_exam_processed)

# Оценка качества моделей

acc_log = accuracy_score(y_test_stroke, y_pred_stroke_log)
f1_log = f1_score(y_test_stroke, y_pred_stroke_log)
roc_auc_log = roc_auc_score(y_test_stroke, y_pred_proba_stroke_log)

print("=== Классификация ===")
print(f"Accuracy: {acc_log:.4f}")
print(f"F1-Score: {f1_log:.4f}")
print(f"ROC-AUC:  {roc_auc_log:.4f}")
print()

mae_lin = mean_absolute_error(y_test_exam, y_pred_exam_lin)
rmse_lin = np.sqrt(mean_squared_error(y_test_exam, y_pred_exam_lin))
r2_lin = r2_score(y_test_exam, y_pred_exam_lin)

print("=== Регрессия ===")
print(f"MAE:  {mae_lin:.4f}")
print(f"RMSE: {rmse_lin:.4f}")
print(f"R²:   {r2_lin:.4f}")

=== Классификация ===
Accuracy: 0.9521
F1-Score: 0.0392
ROC-AUC:  0.8417

=== Регрессия ===
MAE:  5.8876
RMSE: 7.2979
R²:   0.8188


### **Анализ результатов**

**Логистическая регрессия** показала Accuracy 95.2% и высокий ROC-AUC 0.842, что свидетельствует о хорошей разделительной способности модели, однако крайне низкий F1-Score 0.039 подтверждает серьёзную проблему с дисбалансом классов — модель правильно предсказывает отсутствие инсульта, но плохо выявляет положительные случаи, что критично для медицинской задачи.

**Линейная регрессия** продемонстрировала отличные результаты с R²=0.819, объясняя 82% дисперсии данных, при этом MAE=5.89 и RMSE=7.30 указывают на среднюю ошибку менее 6 баллов, что делает модель эффективной для прогнозирования результатов экзамена, хотя и остаётся пространство для улучшения за счёт нелинейных зависимостей и дополнительной обработки признаков.


#Улучшение бейзлайна

Аналогично ЛР 1 пробуем следующие гипотезы для улучшения результатов

-   Балансировка классов
-   Подбор гиперпараметров

- Отбор признаков





In [175]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_stroke_processed, y_train_stroke)
print(f"   До балансировки: {np.bincount(y_train_stroke)}")
print(f"   После балансировки: {np.bincount(y_train_balanced)}")


selector_clf = SelectKBest(f_classif, k=8)
X_train_selected = selector_clf.fit_transform(X_train_balanced, y_train_balanced)
X_test_selected = selector_clf.transform(X_test_stroke_processed)
print(f"   Выбрано {X_train_selected.shape[1]} лучших признаков из {X_train_balanced.shape[1]}")


param_grid_logreg = {
    'C': [0.01, 0.1, 1.0, 10.0, 100.0],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000, 2000]
}

grid_search_logreg = GridSearchCV(
    LogisticRegression(random_state=42, class_weight='balanced'),
    param_grid_logreg,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=0
)
grid_search_logreg.fit(X_train_selected, y_train_balanced)

print(f"   Лучшие параметры: {grid_search_logreg.best_params_}")

best_logreg = grid_search_logreg.best_estimator_
y_pred_logreg_improved = best_logreg.predict(X_test_selected)
y_pred_proba_logreg_improved = best_logreg.predict_proba(X_test_selected)[:, 1]

acc_logreg_improved = accuracy_score(y_test_stroke, y_pred_logreg_improved)
f1_logreg_improved = f1_score(y_test_stroke, y_pred_logreg_improved)
roc_auc_logreg_improved = roc_auc_score(y_test_stroke, y_pred_proba_logreg_improved)

print(f"   Accuracy:  {acc_logreg_improved:.4f}")
print(f"   F1-Score:  {f1_logreg_improved:.4f}")
print(f"   ROC-AUC:   {roc_auc_logreg_improved:.4f}")


   До балансировки: [3889  199]
   После балансировки: [3889 3889]
   Выбрано 8 лучших признаков из 16
   Лучшие параметры: {'C': 0.01, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear'}
   Accuracy:  0.7290
   F1-Score:  0.2284
   ROC-AUC:   0.8408


In [174]:

selector_reg = SelectKBest(f_regression, k=10)
X_train_exam_selected = selector_reg.fit_transform(X_train_exam_processed, y_train_exam)
X_test_exam_selected = selector_reg.transform(X_test_exam_processed)
print(f"   Выбрано {X_train_exam_selected.shape[1]} лучших признаков из {X_train_exam_processed.shape[1]}")


param_grid_reg = {
    'fit_intercept': [True, False],
    'positive': [True, False],
    'copy_X': [True, False]
}

grid_search_reg = GridSearchCV(
    LinearRegression(),
    param_grid_reg,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=0
)
grid_search_reg.fit(X_train_exam_selected, y_train_exam)

print(f"   Лучшие параметры: {grid_search_reg.best_params_}")


best_lin_reg = grid_search_reg.best_estimator_
y_pred_exam_improved = best_lin_reg.predict(X_test_exam_selected)


mae_improved = mean_absolute_error(y_test_exam, y_pred_exam_improved)
rmse_improved = np.sqrt(mean_squared_error(y_test_exam, y_pred_exam_improved))
r2_improved = r2_score(y_test_exam, y_pred_exam_improved)

print(f"   MAE:   {mae_improved:.4f}")
print(f"   RMSE:  {rmse_improved:.4f}")
print(f"   R²:    {r2_improved:.4f}")


   Выбрано 10 лучших признаков из 23
   Лучшие параметры: {'copy_X': True, 'fit_intercept': True, 'positive': False}
   MAE:   6.4801
   RMSE:  8.0722
   R²:    0.7783


 **Сравнение с базовыми результатами**

Логистическая регрессия после балансировки классов и отбора признаков показала значительное улучшение: F1-Score вырос с 0.0392 до 0.2284 (почти в 6 раз), что указывает на эффективность борьбы с дисбалансом, хотя Accuracy снизился с 0.9521 до 0.7290 — ожидаемая плата за повышение чувствительности к миноритарному классу. ROC-AUC остался высоким (0.8408), подтверждая хорошую разделительную способность модели.

Линейная регрессия с отобранными признаками ухудшила все метрики. Отбор только 10 из 23 признаков привёл к потере важной информации для линейной регрессии. Вероятно, удалённые признаки содержали значимые для предсказания зависимости, или линейная модель требует полного набора признаков для построения оптимальных весов. Для линейной регрессии в этом случае отбор признаков оказался контрпродуктивным — лучше использовать все доступные признаки.

#Имплементация собвственной модели


In [176]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iter=1000):
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iter):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict_proba(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        return self._sigmoid(linear_model)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

class MyLinearRegression:
    def __init__(self):
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        X_b = np.c_[np.ones(X.shape[0]), X]
        theta = np.linalg.pinv(X_b.T @ X_b) @ X_b.T @ y
        self.bias = theta[0]
        self.weights = theta[1:]

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

my_log_reg = MyLogisticRegression(learning_rate=0.1, n_iter=2000)
my_log_reg.fit(X_train_stroke_processed, y_train_stroke)
y_pred_my_log = my_log_reg.predict(X_test_stroke_processed)
y_pred_proba_my_log = my_log_reg.predict_proba(X_test_stroke_processed)

my_lin_reg = MyLinearRegression()
my_lin_reg.fit(X_train_exam_selected, y_train_exam)
y_pred_my_lin = my_lin_reg.predict(X_test_exam_selected)

acc_my_log = accuracy_score(y_test_stroke, y_pred_my_log)
f1_my_log = f1_score(y_test_stroke, y_pred_my_log)
roc_auc_my_log = roc_auc_score(y_test_stroke, y_pred_proba_my_log)

print("Логистическая регрессия:")
print(f"Accuracy: {acc_my_log:.4f}")
print(f"F1-Score: {f1_my_log:.4f}")
print(f"ROC-AUC:  {roc_auc_my_log:.4f}")

mae_my_lin = mean_absolute_error(y_test_exam, y_pred_my_lin)
rmse_my_lin = np.sqrt(mean_squared_error(y_test_exam, y_pred_my_lin))
r2_my_lin = r2_score(y_test_exam, y_pred_my_lin)

print("\nЛинейная регрессия:")
print(f"MAE:  {mae_my_lin:.4f}")
print(f"RMSE: {rmse_my_lin:.4f}")
print(f"R²:   {r2_my_lin:.4f}")


Логистическая регрессия:
Accuracy: 0.9511
F1-Score: 0.0385
ROC-AUC:  0.8313

Линейная регрессия:
MAE:  6.4801
RMSE: 8.0722
R²:   0.7783


 Имплементация корректна - результаты близки к sklearn. Введём улучшения

In [179]:
imp_my_knn_clf = MyLogisticRegression()
imp_my_knn_clf.fit(X_train_selected, y_train_balanced)
y_pred_my_imp_clf = imp_my_knn_clf.predict(X_test_selected)

imp_my_knn_reg = MyLinearRegression()
imp_my_knn_reg.fit(X_train_exam_selected, y_train_exam)
y_pred_my_imp_reg = imp_my_knn_reg.predict(X_test_exam_selected)


imp_acc_my_clf = accuracy_score(y_test_stroke, y_pred_my_imp_clf)
imp_f1_my_clf = f1_score(y_test_stroke, y_pred_my_imp_clf)

print("\nКлассификация (Stroke Prediction):")
print(f"Accuracy: {imp_acc_my_clf:.4f}")
print(f"F1-Score: {imp_f1_my_clf:.4f}")

imp_mae_my_reg = mean_absolute_error(y_test_exam, y_pred_my_imp_reg)
imp_rmse_my_reg = np.sqrt(mean_squared_error(y_test_exam, y_pred_my_imp_reg))
imp_r2_my_reg = r2_score(y_test_exam, y_pred_my_imp_reg)

print("\nРегрессия (Exam Performance):")
print(f"MAE:  {imp_mae_my_reg:.4f}")
print(f"RMSE: {imp_rmse_my_reg:.4f}")
print(f"R²:   {imp_r2_my_reg:.4f}")


Классификация (Stroke Prediction):
Accuracy: 0.7133
F1-Score: 0.2187

Регрессия (Exam Performance):
MAE:  6.4801
RMSE: 8.0722
R²:   0.7783


Результаты с улучшением также корректны

#Вывод
Логистическая регрессия показала высокий ROC-AUC (0.841), что свидетельствует о хорошей разделительной способности модели, но низкий F1-Score (0.039) выявил критическую проблему с дисбалансом классов, которая была частично решена применением SMOTE и отбором признаков, повысившим F1 до 0.228; линейная регрессия продемонстрировала отличное качество с R²=0.819, однако отбор признаков незначительно ухудшил метрики (R²=0.778), что указывает на важность сохранения полного набора признаков для линейных моделей, в то время как имплементированные версии обоих алгоритмов подтвердили корректность математических основ, показав результаты, близкие к библиотечным реализациям.